In [1]:
import tensorflow as tf
import os
import numpy as np
import cv2
import random

In [ ]:
train = []
for root,sub,file in os.walk("dogs-vs-cats/train/train"):
    for file_name in file:
        train.append(root+"/"+file_name)
random.shuffle(train)

In [ ]:
def Model():

    Layer_1 = tf.placeholder(name="Input", shape=(None,64,64,3), dtype="float32")
    
    Layer_2 = tf.layers.conv2d(inputs=Layer_1, kernel_size=(3,3), strides=(2,2), filters=20)
    Layer_3 = tf.layers.dropout(inputs=Layer_2, rate=0.2)
    Layer_4 = tf.layers.conv2d(inputs=Layer_3, kernel_size=(1,1), strides=(1,1), filters=30)
    Layer_5 = tf.nn.relu(features=Layer_4)
    Layer_6 = tf.layers.batch_normalization(inputs=Layer_5)
    
    Layer_7 = tf.layers.conv2d(inputs=Layer_6, kernel_size=(3,3), strides=(2,2), filters=40)
    Layer_8 = tf.layers.dropout(inputs=Layer_7,rate=0.2)
    Layer_9 = tf.layers.conv2d(inputs=Layer_8, kernel_size=(1,1), strides=(1,1), filters=40)
    Layer_10 = tf.nn.relu(features=Layer_9)
    Layer_11 = tf.layers.batch_normalization(inputs=Layer_10)

    Layer_12 = tf.layers.conv2d(inputs=Layer_11, kernel_size=(3,3), strides=(2,2), filters=50)
    Layer_13 = tf.layers.dropout(inputs=Layer_12,rate=0.2)
    Layer_14 = tf.layers.conv2d(inputs=Layer_13, kernel_size=(1,1), strides=(1,1), filters=50)
    Layer_15 = tf.nn.relu(features=Layer_14)
    Layer_16 = tf.layers.batch_normalization(inputs=Layer_15)
    
    Layer_17 = tf.layers.conv2d(inputs=Layer_16, kernel_size=(7,7), strides=1, filters=2)
    Layer_18 = tf.reshape(tensor=Layer_17, shape=(-1,2))
#     Layer_18 = tf.squeeze(Layer_17, [1,2])
    
    return Layer_1, Layer_18

In [ ]:
model_input, model_output = Model()

In [ ]:
model_output

In [ ]:
# for variables in tf.all_variables():
#     print("Name  : ", variables.name)
#     print("Shape : ", variables.shape)
#     print("--------------------------")

In [ ]:
actual_output = tf.placeholder(shape=(None, 2), dtype="float32")

In [ ]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model_output, labels=actual_output))

In [ ]:
# loss = tf.losses.mean_squared_error(out=model_output, onehot_labels=actual_output)

In [ ]:
optimizer = tf.train.AdamOptimizer().minimize(loss)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
label_dict = {"cat":[0,1], "dog":[1,0]}
for i in range(10):
    random.shuffle(train)
    for j in range(0,len(train)-32,32):
        image = []
        label = []
        for k in range(j,j+32):
            img = cv2.imread(train[k])
            img = img/255 - 0.5 
            img = cv2.resize(img, (64,64))
            img = np.array(img).astype("float32")
            image.append(img)
            label.append(label_dict[train[k].split("/")[-1].split(".")[0]])   
        opt_ , loss_val = sess.run([optimizer, loss], feed_dict={model_input : image, actual_output : np.array(label).astype("float32")})
        print(loss_val)
#         break
#     break